In [1]:
from dotenv import load_dotenv
load_dotenv()
import os

In [8]:
import re
import os, json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

In [9]:
os.environ["TAVILY_API_KEY"]

'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'

In [12]:
from langchain_community.tools import TavilySearchResults

# 검색할 쿼리 설정
# query = "스테이크와 어울리는 와인을 추천해주세요."
query = "2025년 3월 31일 서울 날씨와 온도."

# Tavily 검색 도구 초기화 (최대 2개의 결과 반환)
web_search = TavilySearchResults(max_results=2)
 

In [ ]:
# 웹 검색 실행
search_results = web_search.invoke(query)

# 검색 결과 출력
for result in search_results:
    print(result)  
    print("-" * 100) 

In [ ]:
# 도구 속성
print("자료형: ")
print(type(web_search))
print("-"*100)

print("name: ")
print(web_search.name)
print("-"*100)

print("description: ")
pprint(web_search.description)
print("-"*100)

print("schema: ")
pprint(web_search.args_schema.schema())
print("-"*100)

In [ ]:
# a= os.getenv('AZURE_OPENAI_VERSION')

In [10]:
import os
os.environ["OPENAI_API_VERSION"] = os.getenv('AZURE_OPENAI_VERSION')
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv('AZURE_OPENAI_END_POINT')
os.environ["AZURE_OPENAI_API_KEY"] = os.getenv('AZURE_OPENAI_KEY')

In [13]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_deployment=os.getenv('DEPLOYMENT_NAME'),  # or your deployment
    api_version=os.getenv('AZURE_OPENAI_VERSION'),  # or your api version
    temperature=0,
    # max_tokens=40000,
    # timeout=None,
    # max_retries=2,
    # other params...
)

In [ ]:
# from langchain_openai import ChatOpenAI
# 
# # ChatOpenAI 모델 초기화
# llm = ChatOpenAI(model="gpt-4o-mini")
# 
# 웹 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=[web_search])

In [ ]:
# 도구 호출이 필요 없는 LLM 호출을 수행
# query = "커피 한잔 마시기 와 음악듣기 중에 어떤 게 더 좋을까.."
query = "커피 마시지 좋은 곳을 알려해줘"
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

In [ ]:
# 도구 호출이 필요한 LLM 호출을 수행
# query = "스테이크와 어울리는 와인을 추천해주세요."
# query = "오늘 서울의 날씨와 온도를 알려주세요."
query = "2025년 3월 27일 서울의 날씨와 온도는"
# query = "커피 마시지 좋은 곳을 알려해줘"
# query = " 서울 동대문 근처에서 커피 마시지 좋은 곳을 알려해줘"
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

In [ ]:
aiai_msg = ai_msg.tool_calls

In [ ]:
aiai_msg.append({'name': 'tavily_search_results_json',
 'args': {'query': '서울 동대문 근처 커피숍 추천'},
 'id': 'call_q4E8kxH1B5CA9LIb85NBlrml',
 'type': 'tool_call'})

In [ ]:
tool_call = ai_msg.tool_calls[0]
tool_call

In [ ]:
### 방법 1: 직접 도구 호출 처리

# 이 방법은 AI 메시지에서 첫 번째 도구 호출을 가져와 직접 처리한다.
# 'args'를 사용하여 도구를 호출하고 결과를 얻는다.

tool_output = web_search.invoke(tool_call["args"])
print(f"{tool_call['name']} 호출 결과:")
print("-" * 100)
print(tool_output)

In [ ]:
### 방법 2: ToolMessage 객체 생성

# 이 방법은 도구 호출 결과를 사용하여 ToolMessage 객체를 생성한다.
# 도구 호출의 ID와 이름을 포함하여 더 구조화된 메시지를 만든다.

from langchain_core.messages import ToolMessage
tool_message = ToolMessage(
    content=tool_output,
    tool_call_id=tool_call["id"],
    name=tool_call["name"]
)

print(tool_message)

In [ ]:
pprint(tool_message.content)

In [ ]:
### 방법 3: 도구 직접 호출하여 바로 ToolMessage 객체 생성

# 이 방법은 도구를 직접 호출하여 ToolMessage 객체를 생성한다.
# 가장 간단하고 직관적인 방법으로, LangChain의 추상화를 활용한다.

tool_message = web_search.invoke(tool_call)

print(tool_message)


In [ ]:
pprint(tool_message.tool_call_id)

In [ ]:
pprint(tool_message.name)

In [ ]:
pprint(tool_message.content)

In [ ]:
ai_msg.tool_calls

In [ ]:
# batch 실행 - 도구 호출이 여러 개인 경우
aiai_msg
# tool_messages = web_search.batch([tool_call])

tool_messages = web_search.batch(ai_msg.tool_calls)

print(tool_messages)
print("-" * 100)
pprint(tool_messages[0].content)

In [ ]:
# batch 실행 - 도구 호출이 여러 개인 경우

# tool_messages = web_search.batch([tool_call])

tool_messages = web_search.batch(aiai_msg)

print(tool_messages)
print("-" * 100)
pprint(tool_messages[0].content)
print("-" * 100)
pprint(tool_messages[1].content)


In [ ]:
for result in tool_messages:
    print("-" * 100)
    pprint(result.content)

In [ ]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
# llm = ChatOpenAI(model="gpt-4o-mini")
llm = AzureChatOpenAI(
    azure_deployment=os.getenv('DEPLOYMENT_NAME'),  # or your deployment
    api_version=os.getenv('AZURE_OPENAI_VERSION'),  # or your api version
    temperature=0,
    # max_tokens=None,
    # timeout=None,
    # max_retries=2,
    # other params...
)

# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[web_search])

# LLM 체인 생성 (LLM 실행을 위한 정의 체인)
llm_chain = prompt | llm_with_tools



In [ ]:
# 도구 실행 체인 정의 (여기가 워크 플로우??)
@chain
def web_search_chain(user_input: str, config: RunnableConfig):
    # prompt template parameter 
    input_ = {"user_input": user_input}
    # 
    ai_msg = llm_chain.invoke(input_, config=config)
    print("ai_msg: \n", ai_msg)
    print("-"*100)
    tool_msgs = web_search.batch(ai_msg.tool_calls, config=config)
    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


In [ ]:
# 체인 실행
response = web_search_chain.invoke("오늘 모엣샹동 샴페인의 가격은 얼마인가요?")
# 체인 실행
# response = web_search_chain.invoke("오늘 서울의 날씨를 알려줘")
# response = web_search_chain.invoke("오늘 회사를 안가는 방법을 알려줘")

# 응답 출력 
pprint(response.content)


In [19]:
#tool search_web
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool
from typing import List

# Tool 정의 
@tool
def search_web(query: str) -> str:
    """Searches the internet for information that does not exist in the database or for the latest information."""

    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)

    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
        ])

    if len(formatted_docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."

In [15]:
# 도구 속성
print("자료형: ")
print(type(search_web))
print("-"*100)

print("name: ")
print(search_web.name)
print("-"*100)

print("description: ")
pprint(search_web.description)
print("-"*100)

print("schema: ")
pprint(search_web.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
search_web
----------------------------------------------------------------------------------------------------
description: 
('Searches the internet for information that does not exist in the database or '
 'for the latest information.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Searches the internet for information that does not exist in '
                'the database or for the latest information.',
 'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'search_web',
 'type': 'object'}
----------------------------------------------------------------------------------------------------


In [ ]:
query = "스테이크와 어울리는 와인을 추천해주세요.가격대와 인기순위에 따라 top 10개로"
search_result = search_web.invoke(query)

print(search_result)

In [ ]:
# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[search_web])

# 도구 호출이 필요한 LLM 호출을 수행
query = "스테이크와 어울리는 와인을 추천해주세요. 가격대와 인기순위에 따라 top 10개로"
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

In [ ]:
from langchain_anthropic import ChatAnthropic
load_dotenv()
# claude 모델 로드 
llm_sonnet = ChatAnthropic(
    # model="claude-3-5-sonnet-20241022",
     model="claude-3-7-sonnet-20250219",
    temperature=0,
    # max_tokens=200, 
    api_key=os.getenv('ANTHROPIC_API_KEY'),
)

In [ ]:
# LLM에 도구 바인딩하여 추가 
tools=[search_web]

llm_sonnet_with_tools = llm_sonnet.bind_tools(tools)

In [ ]:
# 도구 호출이 필요한 LLM 호출을 수행
query = "스테이크와 어울리는 와인을 추천해주세요. 가격대와 인기순위에 따라 top 10개로"
ai_msg = llm_sonnet_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

In [ ]:
# (4)ToolMessage LLM에 전달하여 답변을 생성하기

from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
# # llm = ChatOpenAI(model="gpt-4o-mini")
# llm = AzureChatOpenAI(
#     azure_deployment=os.getenv('DEPLOYMENT_NAME'),  # or your deployment
#     api_version=os.getenv('AZURE_OPENAI_VERSION'),  # or your api version
#     # temperature=0,
#     # max_tokens=None,
#     # timeout=None,
#     # max_retries=2,
#     # other params...
# )

# LLM에 도구를 바인딩
# llm_sonnet_with_tools.invoke(query)
# llm_with_tools = llm.bind_tools(tools=[web_search])

# LLM 체인 생성 (LLM 실행을 위한 정의 체인)
llm_chain = prompt | llm_sonnet_with_tools


# 도구 실행 체인 정의 (여기가 워크 플로우??)
@chain
def web_search_chain(user_input: str, config: RunnableConfig):
    # prompt template parameter 
    input_ = {"user_input": user_input}
    # 
    ai_msg = llm_chain.invoke(input_, config=config)
    print("ai_msg: \n", ai_msg)
    print("-" * 100)
    tool_msgs = web_search.batch(ai_msg.tool_calls, config=config)
    print("tool_msgs: \n", tool_msgs)
    print("-" * 100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


In [ ]:
# 체인 실행
# response = web_search_chain.invoke("오늘 모엣샹동 샴페인의 가격은 얼마인가요?")
# 체인 실행
# response = web_search_chain.invoke("오늘 서울의 날씨를 알려줘")
query = "스테이크와 어울리는 와인을 추천해주세요. 가격대와 인기순위에 따라 top 10개로"
# query = "오늘 회사를 안가는 방법을 알려줘"

response = web_search_chain.invoke(query)

# 응답 출력 
pprint(response.content)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq

# 기본 LLM
llm_gemini_flash = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)
llm_gemini_pro = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)
llm_groq = ChatGroq(model="llama3-groq-70b-8192-tool-use-preview", temperature=0)

# LLM에 도구 바인딩하여 추가 
tools=[search_web]

gemini_flash_with_tools = llm_gemini_flash.bind_tools(tools)
gemini_pro_with_tools = llm_gemini_pro.bind_tools(tools)
groq_llama3_with_tools = llm_groq.bind_tools(tools)

In [ ]:
# 도구 호출이 필요한 LLM 호출을 수행
query = "스테이크와 어울리는 와인을 추천해주세요."
ai_msg = gemini_flash_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

In [ ]:
# 도구 호출이 필요한 LLM 호출을 수행
query = "스테이크와 어울리는 와인을 추천해주세요."
ai_msg = gemini_pro_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

In [ ]:
# 도구 호출이 필요한 LLM 호출을 수행
query = "스테이크와 어울리는 와인을 추천해주세요."
ai_msg = groq_llama3_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

In [16]:
#tool search_wiki
from langchain_community.document_loaders import WikipediaLoader
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel, Field
from typing import List

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하는 함수 
def search_wiki(input_data: dict) -> List[Document]:
    """Search Wikipedia documents based on user input (query) and return k documents"""
    query = input_data["query"]
    k = input_data.get("k", 2)  
    wiki_loader = WikipediaLoader(query=query, load_max_docs=k, lang="ko")
    wiki_docs = wiki_loader.load()
    return wiki_docs

# 도구 호출에 사용할 입력 스키마 정의 
class WikiSearchSchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")
    k: int = Field(2, description="The number of documents to return (default is 2)")

# RunnableLambda 함수를 사용하여 위키피디아 문서 로더를 Runnable로 변환 
runnable = RunnableLambda(search_wiki)
wiki_search = runnable.as_tool(
    name="wiki_search",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a specified number of documents. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSearchSchema
)

In [ ]:
# 도구 속성
print("자료형: ")
print(type(wiki_search))
print("-"*100)

print("name: ")
print(wiki_search.name)
print("-"*100)

print("description: ")
pprint(wiki_search.description)
print("-"*100)

print("schema: ")
pprint(wiki_search.args_schema.schema())
print("-"*100)

In [ ]:
# 위키 검색 실행
query = "파스타의 유래"
wiki_results = wiki_search.invoke({"query":query})

# 검색 결과 출력
for result in wiki_results:
    print(result)  
    print("-" * 100)  

In [ ]:
# LLM에 도구를 바인딩 (2개의 도구 바인딩)
# llm_sonnet_with_tools = llm_sonnet.bind_tools(tools=[search_web, wiki_search])
llm_with_tools = llm.bind_tools(tools=[search_web, wiki_search])
# 도구 호출이 필요한 LLM 호출을 수행
query = "서울 강남의 유명한 파스타 맛집은 어디인가요? 그리고 파스타의 유래를 알려주세요. "
# ai_msg = llm_sonnet_with_tools.invoke(query)
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

In [ ]:
ai_msg.content[0]['text']

In [ ]:
ai_msg.content[1]


In [ ]:
ai_msg.tool_calls

In [ ]:
# LLM에 도구를 바인딩 (2개의 도구 바인딩)
llm_with_tools = llm.bind_tools(tools=[search_web, wiki_search])

# 도구 호출이 필요한 LLM 호출을 수행
query = "서울 강남의 유명한 파스타 맛집은 어디인가요? 그리고 파스타의 유래를 알려주세요. "
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

In [17]:
# tool wiki summary
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_community.document_loaders import WikipediaLoader

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하고 텍스트로 반환하는 함수 
def wiki_search_and_summarize(input_data: dict):
    wiki_loader = WikipediaLoader(query=input_data["query"], load_max_docs=2, lang="ko")
    wiki_docs = wiki_loader.load()

    formatted_docs =[
        f'<Document source="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
        for doc in wiki_docs
        ]
    
    return formatted_docs

# 요약 프롬프트 템플릿
summary_prompt = ChatPromptTemplate.from_template(
    "Summarize the following text in a concise manner:\n\n{context}\n\nSummary in Korean:"
)

# LLM 및 요약 체인 설정
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# summary_chain = (
#     {"context": RunnableLambda(wiki_search_and_summarize)}
#     | summary_prompt | llm | StrOutputParser() 
# )
summary_chain = (
    {"context": RunnableLambda(wiki_search_and_summarize)}
    # | summary_prompt | llm_sonnet | StrOutputParser() 
        | summary_prompt | llm | StrOutputParser() 
)
# 요약 테스트 "파스타의 유래"
summarized_text = summary_chain.invoke({"query":"이탈리안 파스타의 유래"})
pprint(summarized_text)

('리소토는 이탈리아의 전통 요리로, 쌀을 이용해 만드는 음식 중 하나이다. 주로 전식이나 점심 식사로 제공되며, 다양한 재료를 활용해 여러 '
 '종류로 나뉜다. 북부 이탈리아에서 유래했으며, 15세기부터 쌀을 생산하면서 자연스럽게 탄생했다. 조리법은 쌀을 볶은 후 뜨거운 육수를 '
 '부어 익히는 방식이며, 파르미자노 레자노 치즈를 뿌려 마무리한다. \n'
 '\n'
 '마카롱은 프랑스의 대표적인 과자로, 머랭을 주재료로 하여 달걀 흰자, 설탕, 아몬드 가루 등으로 만든다. 이탈리아에서 유래했으며, '
 '1500년대에 프랑스로 전파되었다. 마카롱은 두 겹의 꼬끄 사이에 크림이나 잼을 넣어 만든다. 마카롱과 비슷한 과자로는 마카룬과 '
 '룩셈부르게를리가 있으며, 각각 다른 재료와 방식으로 만들어진다.')


In [18]:
# tool wiki summary
# 도구 호출에 사용할 입력 스키마 정의 
class WikiSummarySchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")

# as_tool 메소드를 사용하여 도구 객체로 변환
wiki_summary = summary_chain.as_tool(
    name="wiki_summary",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a summarized text. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSummarySchema
)

# 도구 속성
print("자료형: ")
print(type(wiki_summary))
print("-"*100)

print("name: ")
print(wiki_summary.name)
print("-"*100)

print("description: ")
pprint(wiki_summary.description)
print("-"*100)

print("schema: ")
pprint(wiki_summary.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
wiki_summary
----------------------------------------------------------------------------------------------------
description: 
('Use this tool when you need to search for information on Wikipedia.\n'
 "It searches for Wikipedia articles related to the user's query and returns\n"
 'a summarized text. This tool is useful when general knowledge\n'
 'or background information is required.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Input schema for Wikipedia search.',
 'properties': {'query': {'description': 'The query to search for in Wikipedia',
                          'title': 'Query',
                          'type': 'string'}},
 'required': ['query'],
 'title': 'WikiSummarySchema',
 'type': 'object'}
-----------------------------

In [ ]:
# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[search_web, wiki_summary])

# 도구 호출이 필요한 LLM 호출을 수행
query = "서울 강남의 유명한 파스타 맛집은 어디인가요? 그리고 파스타의 유래를 알려주세요. "
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

In [ ]:
ai_msg.tool_calls[1]

In [ ]:
# 도구 실행 
tool_message = wiki_summary.invoke(ai_msg.tool_calls[1])

print(tool_message)
print("-" * 100)
pprint(tool_message.content)

In [ ]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[wiki_summary])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def wiki_summary_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    print("ai_msg: \n", ai_msg)
    print("-"*100)
    tool_msgs = wiki_summary.batch(ai_msg.tool_calls, config=config)
    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = wiki_summary_chain.invoke("파스타의 유래에 대해서 알려주세요.")

# 응답 출력 
pprint(response.content)

In [20]:
from langchain.document_loaders import TextLoader
from langchain_core.documents import Document

In [23]:


# 메뉴판 텍스트 데이터를 로드
loader = TextLoader("./data/restaurant_menu.txt", encoding="utf-8")
documents = loader.load()

print(len(documents))

1


In [26]:


# 문서 분할 (Chunking)
def split_menu_items(document):
    """
    메뉴 항목을 분리하는 함수 
    """
    # 정규표현식 정의 
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)
    
    # 각 메뉴 항목을 Document 객체로 변환
    menu_documents = []
    for i, item in enumerate(menu_items, 1):
        # 메뉴 이름 추출
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()
        
        # 새로운 Document 객체 생성
        menu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata['source'],
                "menu_number": i,
                "menu_name": menu_name
            }
        )
        menu_documents.append(menu_doc)
    
    return menu_documents

# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)


In [ ]:
# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")

In [38]:
# Chroma Vectorstore를 사용하기 위한 준비
from langchain_chroma import Chroma
from langchain_ollama  import OllamaEmbeddings
embeddings_model = OllamaEmbeddings(model="bge-m3") 

In [37]:


# embeddings_model = OllamaEmbeddings(model="gemma3") 

# Chroma 인덱스 생성
menu_db = Chroma.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model,   
    collection_name="restaurant_menu",
    persist_directory="./chroma_db",
)

# Retriever 생성
menu_retriever = menu_db.as_retriever(
    search_kwargs={'k': 2},
)



In [ ]:
# 쿼리 테스트
query = "시그니처 스테이크의 가격과 특징은 무엇인가요?"
docs = menu_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

In [28]:
# 와인 메뉴 텍스트 데이터를 로드
loader = TextLoader("./data/restaurant_wine.txt", encoding="utf-8")
documents = loader.load()

# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# # 결과 출력
# print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
# for doc in menu_documents[:2]:
#     print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
#     print(f"메뉴 이름: {doc.metadata['menu_name']}")
#     print(f"내용:\n{doc.page_content[:100]}...")


# Chroma 인덱스 생성
wine_db = Chroma.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model,   
    collection_name="restaurant_wine",
    persist_directory="./chroma_db",
)

wine_retriever = wine_db.as_retriever(
    search_kwargs={'k': 2},
)


In [ ]:

query = "스테이크와 어울리는 와인을 추천해주세요."
docs = wine_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

In [29]:
# 벡터 저장소 로드 tool search menu
menu_db = Chroma(
    embedding_function=embeddings_model,   
    collection_name="restaurant_menu",
    persist_directory="./chroma_db",
)

@tool
def search_menu(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized restaurant menu information from the encrypted database.
    Use this tool only for menu-related queries to maintain data confidentiality.
    """
    docs = menu_db.similarity_search(query, k=2)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]



In [ ]:
# 도구 속성
print("자료형: ")
print(type(search_menu))
print("-"*100)

print("name: ")
print(search_menu.name)
print("-"*100)

print("description: ")
pprint(search_menu.description)
print("-"*100)

print("schema: ")
pprint(search_menu.args_schema.schema())
print("-"*100)

In [30]:
from langchain_core.tools import tool
from typing import List
from langchain_core.documents import Document

# 벡터 저장소 로드
wine_db = Chroma(
   embedding_function=embeddings_model,   
   collection_name="restaurant_wine",
   persist_directory="./chroma_db",
)

@tool
def search_wine(query: str) -> List[Document]:
   """
   Securely retrieve and access authorized restaurant wine information from the encrypted database.
   Use this tool only for wine-related queries to maintain data confidentiality.
   """
   docs = wine_db.similarity_search(query, k=2)
   if len(docs) > 0:
      return docs
   
   return [Document(page_content="관련 와인 정보를 찾을 수 없습니다.")]


In [ ]:

# 도구 속성
print("자료형: ")
print(type(search_wine))
print("-"*100)

print("name: ")
print(search_wine.name)
print("-"*100)

print("description: ")
pprint(search_wine.description)
print("-"*100)

print("schema: ")
pprint(search_wine.args_schema.schema())
print("-"*100)

In [ ]:
# LLM에 도구를 바인딩 (2개의 도구 바인딩)
llm_with_tools = llm.bind_tools(tools=[search_menu, search_wine])

# 도구 호출이 필요한 LLM 호출을 수행
query = "시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요."
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

In [31]:
tools = [search_web, wiki_summary, search_wine, search_menu]
for tool in tools:
    print(tool.name)
    
    

search_web
wiki_summary
search_wine
search_menu


In [ ]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
# llm = ChatOpenAI(model="gpt-4o-mini")

# 4개의 검색 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools=tools)

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def restaurant_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        if tool_call["name"] == "search_web":
            tool_message = search_web.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "search_wine":
            tool_message = search_wine.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "search_menu":
            tool_message = search_menu.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)            

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = restaurant_menu_chain.invoke("시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요.")

# 응답 출력 
print(response.content)

In [ ]:
# 체인 실행
response = restaurant_menu_chain.invoke("파스타 메뉴가 있나요? 이 음식의 역사 또는 유래를 알려주세요.")

# 응답 출력 
print(response.content)

In [ ]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
# llm = ChatOpenAI(model="gpt-4o-mini")

# 4개의 검색 도구를 LLM에 바인딩
llm_with_tools = llm_sonnet.bind_tools(tools=tools)

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
# ai_msg.content[0]['text']
# ai_msg.content[1]
# 
# ai_msg.tool_calls
input_ = {"user_input": "시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요."}
ai_msg = llm_chain.invoke(input_)
print ("ai_msgs: \n", ai_msg)
print("-"*100)

In [ ]:

@chain
def restaurant_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    print("ai_msgs: \n", ai_msg)
    if ai_msg.content[0]:
        print ("ai_msgs: \n", ai_msg)
        print("-"*100)
        print ("Asisstant: \n",ai_msg.content[0]['text'])
        print("-"*100)
    if ai_msg.content[1]:
        tool_msgs = []
        for tool_call in ai_msg.tool_calls:
            print(f"{tool_call['name']}: \n{tool_call}")
            print("-"*100)
    
            if tool_call["name"] == "search_web":
                tool_message = search_web.invoke(tool_call, config=config)
                tool_msgs.append(tool_message)
    
            elif tool_call["name"] == "wiki_summary":
                tool_message = wiki_summary.invoke(tool_call, config=config)
                tool_msgs.append(tool_message)
    
            elif tool_call["name"] == "search_wine":
                tool_message = search_wine.invoke(tool_call, config=config)
                tool_msgs.append(tool_message)
    
            elif tool_call["name"] == "search_menu":
                tool_message = search_menu.invoke(tool_call, config=config)
                tool_msgs.append(tool_message)            
    
        print("tool_msgs: \n", tool_msgs)
        print("-"*100)
        return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = restaurant_menu_chain.invoke("시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요.")

# 응답 출력 
print(response.content)

In [ ]:
# 체인 실행
response = restaurant_menu_chain.invoke("파스타 메뉴가 있나요? 이 음식의 역사 또는 유래를 알려주세요.")

# 응답 출력 
print(response.content)

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate

examples = [
    HumanMessage("트러플 리조또의 가격과 특징, 그리고 어울리는 와인에 대해 알려주세요.", name="example_user"),
    AIMessage("메뉴 정보를 검색하고, 위키피디아에서 추가 정보를 찾은 후, 어울리는 와인을 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "search_menu", "args": {"query": "트러플 리조또"}, "id": "1"}]),
    ToolMessage("트러플 리조또: 가격 ₩28,000, 이탈리아 카나롤리 쌀 사용, 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리", tool_call_id="1"),
    AIMessage("트러플 리조또의 가격은 ₩28,000이며, 이탈리아 카나롤리 쌀을 사용하고 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리합니다. 이제 추가 정보를 위키피디아에서 찾아보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "wiki_summary", "args": {"query": "트러플 리조또", "k": 1}, "id": "2"}]),
    ToolMessage("트러플 리조또는 이탈리아 요리의 대표적인 리조또 요리 중 하나로, 고급 식재료인 트러플을 사용하여 만든 크리미한 쌀 요리입니다. 주로 아르보리오나 카나롤리 등의 쌀을 사용하며, 트러플 오일이나 생 트러플을 넣어 조리합니다. 리조또 특유의 크리미한 질감과 트러플의 강렬하고 독특한 향이 조화를 이루는 것이 특징입니다.", tool_call_id="2"),
    AIMessage("트러플 리조또의 특징에 대해 알아보았습니다. 이제 어울리는 와인을 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "search_wine", "args": {"query": "트러플 리조또에 어울리는 와인"}, "id": "3"}]),
    ToolMessage("트러플 리조또와 잘 어울리는 와인으로는 주로 중간 바디의 화이트 와인이 추천됩니다. 1. 샤르도네: 버터와 오크향이 트러플의 풍미를 보완합니다. 2. 피노 그리지오: 산뜻한 산미가 리조또의 크리미함과 균형을 이룹니다. 3. 베르나차: 이탈리아 토스카나 지방의 화이트 와인으로, 미네랄리티가 트러플과 잘 어울립니다.", tool_call_id="3"),
    AIMessage("트러플 리조또(₩28,000)는 이탈리아의 대표적인 리조또 요리 중 하나로, 이탈리아 카나롤리 쌀을 사용하고 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리합니다. 주요 특징으로는 크리미한 질감과 트러플의 강렬하고 독특한 향이 조화를 이루는 점입니다. 고급 식재료인 트러플을 사용해 풍부한 맛과 향을 내며, 주로 아르보리오나 카나롤리 등의 쌀을 사용합니다. 트러플 리조또와 잘 어울리는 와인으로는 중간 바디의 화이트 와인이 추천됩니다. 특히 버터와 오크향이 트러플의 풍미를 보완하는 샤르도네, 산뜻한 산미로 리조또의 크리미함과 균형을 이루는 피노 그리지오, 그리고 미네랄리티가 트러플과 잘 어울리는 이탈리아 토스카나 지방의 베르나차 등이 좋은 선택이 될 수 있습니다.", name="example_assistant"),
]

system = """You are an AI assistant providing restaurant menu information and general food-related knowledge.
For information about the restaurant's menu, use the search_menu tool.
For other general information, use the wiki_summary tool.
For wine recommendations or pairing information, use the search_wine tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    *examples,
    ("human", "{query}"),
])

# ChatOpenAI 모델 초기화 
# llm = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

# 체인 실행
query = "스테이크 메뉴가 있나요? 스테이크와 어울리는 와인을 추천해주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

In [ ]:
# 체인 실행
query = "파스타의 유래에 대해서 알고 있나요? 서울 강남의 파스타 맛집을 추천해주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

In [ ]:
from datetime import datetime
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_openai import ChatOpenAI

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
system = """You are an AI assistant providing restaurant menu information and general food-related knowledge.
For information about the restaurant's menu, use the search_menu tool.
For other general information, use the wiki_summary tool.
For wine recommendations or pairing information, use the search_wine tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system + f"Today's date is {today}."),
    *examples,
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
# llm = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def restaurant_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        if tool_call["name"] == "search_web":
            tool_message = search_web.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "search_wine":
            tool_message = search_wine.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "search_menu":
            tool_message = search_menu.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)            

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return fewshot_search_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


# 체인 실행
query = "스테이크 메뉴가 있나요? 스테이크와 어울리는 와인을 추천해주세요."
response = restaurant_menu_chain.invoke(query)

# 응답 출력 
pprint(response.content)

In [39]:
# 체인 실행
query = "파스타의 유래에 대해서 알고 있나요? 서울 강남의 파스타 맛집을 추천해주세요."
response = restaurant_menu_chain.invoke(query)

# 응답 출력 
pprint(response.content)


KeyboardInterrupt



In [32]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

agent_prompt = ChatPromptTemplate.from_messages([
    ("system", dedent("""
        You are an AI assistant providing restaurant menu information and general food-related knowledge. 
        Your main goal is to provide accurate information and effective recommendations to users.

        Key guidelines:
        1. For restaurant menu information, use the search_menu tool. This tool provides details on menu items, including prices, ingredients, and cooking methods.
        2. For general food information, history, and cultural background, utilize the wiki_summary tool.
        3. For wine recommendations or food and wine pairing information, use the search_wine tool.
        4. If additional web searches are needed or for the most up-to-date information, use the search_web tool.
        5. Provide clear and concise responses based on the search results.
        6. If a question is ambiguous or lacks necessary information, politely ask for clarification.
        7. Always maintain a helpful and professional tone.
        8. When providing menu information, describe in the order of price, main ingredients, and distinctive cooking methods.
        9. When making recommendations, briefly explain the reasons.
        10. Maintain a conversational, chatbot-like style in your final responses. Be friendly, engaging, and natural in your communication.


        Remember, understand the purpose of each tool accurately and use them in appropriate situations. 
        Combine the tools to provide the most comprehensive and accurate answers to user queries. 
        Always strive to provide the most current and accurate information.
        """)),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [34]:
# Tool calling Agent 생성
from langchain.agents import AgentExecutor, create_tool_calling_agent

tools = [search_web, wiki_summary, search_wine, search_menu]
agent = create_tool_calling_agent(llm, tools, agent_prompt)

# AgentExecutor 생성 
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [35]:
# AgentExecutor 실행

query = "시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요."
agent_response = agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...

Invoking: `search_menu` with `{'query': '시그니처 스테이크'}`


[Document(metadata={'menu_name': '시그니처 스테이크', 'menu_number': 1, 'source': './data/restaurant_menu.txt'}, page_content='1. 시그니처 스테이크\n   • 가격: ₩35,000\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.'), Document(metadata={'menu_name': '시그니처 스테이크', 'menu_number': 1, 'source': './data/restaurant_menu.txt'}, page_content='1. 시그니처 스테이크\n   • 가격: ₩35,000\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.')]
Invoking: `search_wine` with `{'query': '스테이크 와인 추천'}`


[Document(metadata={'menu_name': '사시카이아 2018', 'menu_number': 3, 'source': './data/restaurant_wine.txt'}, page_content='3. 사시카이아 2018\n   • 가격: ₩420,00

In [36]:
pprint(agent_response)

{'input': '시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요.',
 'output': '시그니처 스테이크에 대해 알려드릴게요!\n'
           '\n'
           '### 시그니처 스테이크\n'
           '- **가격**: ₩35,000\n'
           '- **주요 식재료**: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\n'
           '- **설명**: 이 스테이크는 21일간 건조 숙성한 최상급 한우 등심을 사용하여, 미디엄 레어로 조리해 육즙을 최대한 '
           '보존합니다. 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여져 풍부한 맛을 더합니다. 레드와인 소스와 함께 '
           '제공되어 더욱 깊은 맛을 느낄 수 있습니다.\n'
           '\n'
           '### 와인 추천\n'
           '- **사시카이아 2018**\n'
           '  - **가격**: ₩420,000\n'
           '  - **주요 품종**: 카베르네 소비뇽, 카베르네 프랑, 메를로\n'
           '  - **설명**: 이탈리아 토스카나의 슈퍼 투스칸 와인으로, 블랙베리, 카시스의 강렬한 과실향과 함께 허브, 가죽, '
           '스파이스 노트가 복잡성을 더합니다. 풀바디이지만 우아한 타닌과 신선한 산도가 균형을 잡아줍니다. 오크 숙성으로 인한 '
           '바닐라, 초콜릿 향이 은은하게 느껴집니다.\n'
           '\n'
           '시그니처 스테이크와 사시카이아 2018 와인은 서로의 풍미를 극대화시켜주는 훌륭한 조합입니다. 맛있게 즐기세요!'}


In [ ]:
import gradio as gr
from typing import List, Tuple
import logging

def answer_invoke(message: str, history: List[Tuple[str, str]]) -> str:
    try:
        # 채팅 기록을 AI에게 전달할 수 있는 형식으로 변환
        chat_history = []
        for human, ai in history:
            chat_history.append(HumanMessage(content=human))
            chat_history.append(AIMessage(content=ai))
        
        # agent_executor를 사용하여 응답 생성
        response = agent_executor.invoke({
            "input": message,
            "chat_history": chat_history[-4:]    # 최근 2개의 메시지 기록만을 활용 
        })
        
        # agent_executor의 응답에서 최종 답변 추출
        return response['output']
    except Exception as e:
        # 오류 발생 시 사용자에게 알리고 로그 기록
        print(f"Error occurred: {str(e)}")
        return "죄송합니다. 응답을 생성하는 동안 오류가 발생했습니다. 다시 시도해 주세요."

# 예제 질문 정의
example_questions = [
    "시그니처 스테이크의 가격과 특징을 알려주세요.",
    "트러플 리조또와 잘 어울리는 와인을 추천해주세요.",
    "해산물 파스타의 주요 재료는 무엇인가요? 서울 강남 지역에 레스토랑을 추천해주세요.",
    "채식주의자를 위한 메뉴 옵션이 있나요?"
]

# Gradio 인터페이스 생성
demo = gr.ChatInterface(
    fn=answer_invoke,
    title="레스토랑 메뉴 AI 어시스턴트",
    description="메뉴 정보, 추천, 음식 관련 질문에 답변해 드립니다.",
    examples=example_questions,
    theme=gr.themes.Soft()
)



In [ ]:
# 데모 실행
demo.launch()

In [ ]:
# 데모 종료
demo.close()

In [ ]:
import gradio as gr
from typing import List, Tuple
import logging

# 설정할 로깅 레벨과 포맷 정의
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def answer_invoke(message: str, history: List[Tuple[str, str]]) -> str:
    try:
        logger.debug(f"Received message: {message}")
        logger.debug(f"Chat history: {history}")

        # 채팅 기록을 AI에게 전달할 수 있는 형식으로 변환
        chat_history = []
        for human, ai in history:
            chat_history.append(HumanMessage(content=human))
            chat_history.append(AIMessage(content=ai))
        
        logger.debug(f"Formatted chat history: {chat_history[-4:]}")
        
        # agent_executor를 사용하여 응답 생성
        response = agent_executor.invoke({
            "input": message,
            "chat_history": chat_history[-4:]    # 최근 2개의 메시지 기록만을 활용 
        })
        
        logger.debug(f"Response from agent_executor: {response}")
        
        # agent_executor의 응답에서 최종 답변 추출
        return response['output']
    except Exception as e:
        logger.error(f"Error occurred: {str(e)}")
        return "죄송합니다. 응답을 생성하는 동안 오류가 발생했습니다. 다시 시도해 주세요."

# 예제 질문 정의
example_questions = [
    "시그니처 스테이크의 가격과 특징을 알려주세요.",
    "트러플 리조또와 잘 어울리는 와인을 추천해주세요.",
    "해산물 파스타의 주요 재료는 무엇인가요? 서울 강남 지역에 레스토랑을 추천해주세요.",
    "채식주의자를 위한 메뉴 옵션이 있나요?"
]

# Gradio 인터페이스 생성
demo = gr.ChatInterface(
    fn=answer_invoke,
    title="레스토랑 메뉴 AI 어시스턴트",
    description="메뉴 정보, 추천, 음식 관련 질문에 답변해 드립니다.",
    examples=example_questions,
    theme=gr.themes.Soft()
)
